# HW-7

## Part-1

In [1]:
!curl -L -o cifake.zip https://www.kaggle.com/api/v1/datasets/download/birdy654/cifake-real-and-ai-generated-synthetic-images

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  104M  100  104M    0     0   123M      0 --:--:-- --:--:-- --:--:--  123M


In [2]:
import zipfile
zipfile.ZipFile("cifake.zip", "r").extractall("cifake")

In [3]:
import os, shutil, random

base_dir = "cifake/train"
val_dir = "cifake/validation"
os.makedirs(f"{val_dir}/REAL", exist_ok=True)
os.makedirs(f"{val_dir}/FAKE", exist_ok=True)

for category in ['REAL', 'FAKE']:
    src_dir = os.path.join(base_dir, category)
    dest_dir = os.path.join(val_dir, category)
    files = os.listdir(src_dir)
    random.shuffle(files)
    for f in files[:10000]:
        shutil.move(os.path.join(src_dir, f), os.path.join(dest_dir, f))


## Part-2

In [4]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    "cifake/train", image_size=(32, 32), batch_size=64)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "cifake/validation", image_size=(32, 32), batch_size=64)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "cifake/test", image_size=(32, 32), batch_size=64)

Found 80000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


In [5]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(32, 32, 3)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,001 (328.13 KB)

 Trainable params: 84,001 (328.13 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_cb = ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [8]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[checkpoint_cb]
)


Epoch 1/30
1242/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7676 - loss: 0.4666
Epoch 1: val_loss improved from inf to 0.28968, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.7680 - loss: 0.4660 - val_accuracy: 0.8798 - val_loss: 0.2897
Epoch 2/30
1247/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8847 - loss: 0.2746
Epoch 2: val_loss did not improve from 0.28968
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.8847 - loss: 0.2745 - val_accuracy: 0.8741 - val_loss: 0.2973
Epoch 3/30
1246/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9123 - loss: 0.2206
Epoch 3: val_loss improved from 0.28968 to 0.20476, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9123 - loss: 0.2206 - val_accuracy: 0.9184 - val_loss: 0.2048
Epoch 4/30
1245/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9227 - loss: 0.1935
Epoch 4: val_loss did not improve from 0.20476
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.9228 - loss: 0.1935 - val_accuracy: 0.9145 - val_loss: 0.2154
Epoch 5/30
1248/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9295 - loss: 0.1806
Epoch 5: val_loss improved from 0.20476 to 0.18103, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9295 - loss: 0.1806 - val_accuracy: 0.9291 - val_loss: 0.1810
Epoch 6/30
1247/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9348 - loss: 0.1647
Epoch 6: val_loss did not improve from 0.18103
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9348 - loss: 0.1647 - val_accuracy: 0.9230 - val_loss: 0.1932
Epoch 7/30
1243/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9402 - loss: 0.1535
Epoch 7: val_loss improved from 0.18103 to 0.17055, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9402 - loss: 0.1535 - val_accuracy: 0.9342 - val_loss: 0.1706
Epoch 8/30
1243/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9414 - loss: 0.1493
Epoch 8: val_loss did not improve from 0.17055
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9414 - loss: 0.1493 - val_accuracy: 0.9313 - val_loss: 0.1770
Epoch 9/30
1246/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9467 - loss: 0.1367
Epoch 9: val_loss did not improve from 0.17055
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.9467 - loss: 0.1368 - val_accuracy: 0.9327 - val_loss: 0.1724
Epoch 10/30
1246/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9496 - loss: 0.1275
Epoch 10: val_loss improved from 0.17055 to 0.16541, saving model to best_model.h5


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.9496 - loss: 0.1275 - val_accuracy: 0.9374 - val_loss: 0.1654
Epoch 11/30
1245/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9537 - loss: 0.1192
Epoch 11: val_loss did not improve from 0.16541
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.9537 - loss: 0.1192 - val_accuracy: 0.9287 - val_loss: 0.1930
Epoch 12/30
1245/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9560 - loss: 0.1124
Epoch 12: val_loss did not improve from 0.16541
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9560 - loss: 0.1124 - val_accuracy: 0.9337 - val_loss: 0.1827
Epoch 13/30
1248/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9595 - loss: 0.1050
Epoch 13: val_loss did not improve from 0.16541
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9595 - loss: 0.1050 - val_accuracy: 0.9414 - val_loss: 0.1663
Epoch 14/30
1244/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9624 - loss: 0.0985
Epoch 14: val_loss d

In [9]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9260 - loss: 0.3674

Test Accuracy: 92.93%
